In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [2]:
# Set view options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
obs = pd.read_csv("../data/obs_windows/observation_21D_prediction_24H_48H_72H.csv")

C:\Users\Conrad\AppData\Local\Temp\ipykernel_21460\3942603523.py:1: DtypeWarning: Columns (3,4,7,8,10,11,12,13,14,15,16,17,19,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  obs = pd.read_csv("../data/obs_windows/observation_21D_prediction_24H_48H_72H.csv")


### 24hr prediction window

In [4]:
obs_24 = obs.drop(obs.loc[:, 'spn':'full_derate_within_24hr'], axis=1).drop(obs.loc[:, 'partial_derate_within_48hr':'derate_within_72hr'], axis=1).drop(columns='EventTimeStamp')

In [5]:
eq_ids = obs_24.drop_duplicates(subset='EquipmentID')
eq_ids = eq_ids[['EquipmentID']]
eq_ids.shape[0]

1039

In [6]:
np.random.seed(321)

eq_ids['random'] = (np.random.randint(0, 10000, eq_ids.shape[0]))/10000

In [7]:
#create column 'modeling' labeling ~35% of the data for training, ~30% for validation, and ~35% for testing 
eq_ids['modeling'] = np.where(((eq_ids.random <= 0.35)), 'training', np.where(((eq_ids.random <= 0.65)), 'validation', 'testing'))

In [8]:
#merge modeling column onto original dataframe
obs_24 = obs_24.sort_values(by='EquipmentID')
eq_ids = eq_ids.sort_values(by='EquipmentID')

obs_24 = obs_24.merge(eq_ids, on='EquipmentID')
obs_24 = obs_24.drop(columns='EquipmentID')

In [9]:
#creating training, testing, and validation dfs
obs_24_training = obs_24[obs_24['modeling']=='training']
obs_24_training = obs_24_training.drop(columns=['modeling', 'random'])
print(obs_24_training.shape)

obs_24_test = obs_24[obs_24['modeling']=='testing']
obs_24_test = obs_24_test.drop(columns=['modeling', 'random'])
print(obs_24_test.shape)

obs_24_validation = obs_24[obs_24['modeling']=='validation']
obs_24_validation = obs_24_validation.drop(columns=['modeling', 'random'])
print(obs_24_validation.shape)

(200860, 56)
(202749, 56)
(151246, 56)


In [10]:
df_sampling = obs_24_training
df_sampling_non_tenth = df_sampling[df_sampling['derate_within_24hr'] == 0][0:18855]

In [11]:
df_sampling_derate_tenth = pd.concat([df_sampling[df_sampling['derate_within_24hr']==1], df_sampling_non_tenth])

In [12]:
oversampler = SMOTE(k_neighbors=5, random_state=321)

features = df_sampling_derate_tenth.loc[:, 'Aftertreatment System':].columns.tolist()

target = ['derate_within_24hr']

X_train = df_sampling_derate_tenth[features]
y_train = df_sampling_derate_tenth[target]

In [13]:
X_smote, y_smote = oversampler.fit_resample(X_train, y_train)

In [14]:
smote_all = pd.concat([y_smote, X_smote], axis=1)
smote_derate = smote_all[smote_all['derate_within_24hr']==1]
smote_derate.shape

(18855, 56)

In [15]:
und_samp = pd.concat([smote_derate, df_sampling[df_sampling['derate_within_24hr']==0]])
und_samp.shape

(215061, 56)

In [16]:
undersampler = RandomUnderSampler(random_state=321, sampling_strategy=1/2)

features = und_samp.loc[:, 'Aftertreatment System':].columns.tolist()

target =['derate_within_24hr']

X_train_final = und_samp[features]
y_train_final = und_samp[target]

X_test = obs_24_test[features]
y_test = obs_24_test[target]

In [17]:
X_resampled, y_resampled = undersampler.fit_resample(X_train_final, y_train_final)

In [18]:
print(y_resampled.shape)

(56565, 1)


In [19]:
rf_pipeline = Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('rf_clf', RandomForestClassifier(n_estimators = 500, n_jobs = -1, random_state = 69))
    ]
)

In [20]:
rand_forest = rf_pipeline.fit(X_resampled, y_resampled)
rf_pipeline.score(X_test, y_test)

y_pred = rand_forest.predict(X_test)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


In [21]:
data = {'feature': np.array(X_resampled.columns), 'importance': rf_pipeline.named_steps['rf_clf'].feature_importances_}
pd.DataFrame(data).sort_values('importance', ascending = False)


,feature,importance
29,Engine Protection Torque Partial Derate,3.088825e-01
0,Aftertreatment System,2.141843e-01
51,Unknown,1.394362e-01
18,Engine Coolant,8.119624e-02
22,Engine Fuel,4.257644e-02
43,J1939 Network,3.085046e-02
28,Engine Protection Torque Full Derate,2.877766e-02
41,Intake Manifold,2.615304e-02
52,Variable Geometry Turbocharger,2.187922e-02
17,Engine Control Module,1.747394e-02


In [22]:
confusion_matrix(y_test, y_pred)

array([[191854,   6158],
       [  1625,   3112]], dtype=int64)

In [23]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [24]:
(tn, fp, fn, tp)

(191854, 6158, 1625, 3112)

In [25]:
print(classification_report(y_test, y_pred, zero_division = 0))

              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98    198012
         1.0       0.34      0.66      0.44      4737

    accuracy                           0.96    202749
   macro avg       0.66      0.81      0.71    202749
weighted avg       0.98      0.96      0.97    202749



In [122]:
lg_pipeline =  Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('lg_clf', LogisticRegression())
    ]
)

In [124]:
logistic = lg_pipeline.fit(X_resampled, y_resampled)

y_pred2 = logistic.predict(X_test)

y_pred_proba = logistic.predict_proba(X_test)[::,1]

auc = metrics.roc_auc_score(y_test, y_pred_proba)

print(auc)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9211188136613722


### 48hr prediction window

In [26]:
obs_48 = obs.drop(obs.loc[:, 'spn':'full_derate_within_48hr'], axis=1).drop(obs.loc[:, 'partial_derate_within_72hr':'derate_within_72hr'], axis=1).drop(columns='EventTimeStamp')

In [27]:
eq_ids = obs_48.drop_duplicates(subset='EquipmentID')
eq_ids = eq_ids[['EquipmentID']]
eq_ids.shape[0]

1039

In [28]:
np.random.seed(321)

eq_ids['random'] = (np.random.randint(0, 10000, eq_ids.shape[0]))/10000

In [29]:
#create column 'modeling' labeling ~35% of the data for training, ~30% for validation, and ~35% for testing 
eq_ids['modeling'] = np.where(((eq_ids.random <= 0.35)), 'training', np.where(((eq_ids.random <= 0.65)), 'validation', 'testing'))

In [30]:
#merge modeling column onto original dataframe
obs_48 = obs_48.sort_values(by='EquipmentID')
eq_ids = eq_ids.sort_values(by='EquipmentID')

obs_48 = obs_48.merge(eq_ids, on='EquipmentID')
obs_48 = obs_48.drop(columns='EquipmentID')

In [31]:
#creating training, testing, and validation dfs
obs_48_training = obs_48[obs_48['modeling']=='training']
obs_48_training = obs_48_training.drop(columns=['modeling', 'random'])
print(obs_48_training.shape)

obs_48_test = obs_48[obs_48['modeling']=='testing']
obs_48_test = obs_48_test.drop(columns=['modeling', 'random'])
print(obs_48_test.shape)

obs_48_validation = obs_48[obs_48['modeling']=='validation']
obs_48_validation = obs_48_validation.drop(columns=['modeling', 'random'])
print(obs_48_validation.shape)

(200860, 56)
(202749, 56)
(151246, 56)


In [32]:
df_sampling = obs_48_training
df_sampling_non_tenth = df_sampling[df_sampling['derate_within_48hr'] == 0][0:18855]

In [33]:
df_sampling_derate_tenth = pd.concat([df_sampling[df_sampling['derate_within_48hr']==1], df_sampling_non_tenth])

In [34]:
oversampler = SMOTE(k_neighbors=5, random_state=321)

features = df_sampling_derate_tenth.loc[:, 'Aftertreatment System':].columns.tolist()

target = ['derate_within_48hr']

X_train = df_sampling_derate_tenth[features]
y_train = df_sampling_derate_tenth[target]

X_test = obs_48_test[features]
y_test = obs_48_test[target]

In [35]:
X_smote, y_smote = oversampler.fit_resample(X_train, y_train)

In [36]:
smote_all = pd.concat([y_smote, X_smote], axis=1)
smote_derate = smote_all[smote_all['derate_within_48hr']==1]
smote_derate.shape

(18855, 56)

In [37]:
und_samp = pd.concat([smote_derate, df_sampling[df_sampling['derate_within_48hr']==0]])
und_samp.shape

(214184, 56)

In [38]:
undersampler = RandomUnderSampler(random_state=321, sampling_strategy=1/2)

features = und_samp.loc[:, 'Aftertreatment System':].columns.tolist()

target =['derate_within_48hr']

X_train_final = und_samp[features]
y_train_final = und_samp[target]

In [39]:
X_resampled, y_resampled = undersampler.fit_resample(X_train_final, y_train_final)

In [40]:
print(y_resampled.shape)

(56565, 1)


In [41]:
rf_pipeline = Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('rf_clf', RandomForestClassifier(n_estimators = 500, n_jobs = -1, random_state = 69))
    ]
)

In [42]:
rand_forest = rf_pipeline.fit(X_resampled, y_resampled)
rf_pipeline.score(X_test, y_test)

y_pred = rand_forest.predict(X_test)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


In [43]:
data = {'feature': np.array(X_resampled.columns), 'importance': rf_pipeline.named_steps['rf_clf'].feature_importances_}
pd.DataFrame(data).sort_values('importance', ascending = False)


,feature,importance
29,Engine Protection Torque Partial Derate,2.778945e-01
0,Aftertreatment System,2.135735e-01
51,Unknown,1.494911e-01
18,Engine Coolant,8.952419e-02
22,Engine Fuel,5.016658e-02
43,J1939 Network,3.899121e-02
28,Engine Protection Torque Full Derate,2.509791e-02
41,Intake Manifold,2.435798e-02
52,Variable Geometry Turbocharger,2.087423e-02
17,Engine Control Module,1.850736e-02


In [44]:
confusion_matrix(y_test, y_pred)

array([[190450,   6878],
       [  2230,   3191]], dtype=int64)

In [45]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [46]:
(tn, fp, fn, tp)

(190450, 6878, 2230, 3191)

In [47]:
print(classification_report(y_test, y_pred, zero_division = 0))

              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98    197328
         1.0       0.32      0.59      0.41      5421

    accuracy                           0.96    202749
   macro avg       0.65      0.78      0.69    202749
weighted avg       0.97      0.96      0.96    202749



In [122]:
lg_pipeline =  Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('lg_clf', LogisticRegression())
    ]
)

In [124]:
logistic = lg_pipeline.fit(X_resampled, y_resampled)

y_pred2 = logistic.predict(X_test)

y_pred_proba = logistic.predict_proba(X_test)[::,1]

auc = metrics.roc_auc_score(y_test, y_pred_proba)

print(auc)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9211188136613722


### 7d observation window with 72hr prediction window

In [48]:
obs_72 = obs.drop(obs.loc[:, 'spn':'full_derate_within_72hr'], axis=1).drop(columns='EventTimeStamp')

In [49]:
eq_ids = obs_72.drop_duplicates(subset='EquipmentID')
eq_ids = eq_ids[['EquipmentID']]
eq_ids.shape[0]

1039

In [50]:
np.random.seed(321)

eq_ids['random'] = (np.random.randint(0, 10000, eq_ids.shape[0]))/10000

In [51]:
#create column 'modeling' labeling ~35% of the data for training, ~30% for validation, and ~35% for testing 
eq_ids['modeling'] = np.where(((eq_ids.random <= 0.35)), 'training', np.where(((eq_ids.random <= 0.65)), 'validation', 'testing'))

In [52]:
#merge modeling column onto original dataframe
obs_72 = obs_72.sort_values(by='EquipmentID')
eq_ids = eq_ids.sort_values(by='EquipmentID')

obs_72 = obs_72.merge(eq_ids, on='EquipmentID')
obs_72 = obs_72.drop(columns='EquipmentID')

In [53]:
#creating training, testing, and validation dfs
obs_72_training = obs_72[obs_72['modeling']=='training']
obs_72_training = obs_72_training.drop(columns=['modeling', 'random'])
print(obs_72_training.shape)

obs_72_test = obs_72[obs_72['modeling']=='testing']
obs_72_test = obs_72_test.drop(columns=['modeling', 'random'])
print(obs_72_test.shape)

obs_72_validation = obs_72[obs_72['modeling']=='validation']
obs_72_validation = obs_72_validation.drop(columns=['modeling', 'random'])
print(obs_72_validation.shape)

(200860, 56)
(202749, 56)
(151246, 56)


In [54]:
df_sampling = obs_72_training
df_sampling_non_tenth = df_sampling[df_sampling['derate_within_72hr'] == 0][0:18855]

In [55]:
df_sampling_derate_tenth = pd.concat([df_sampling[df_sampling['derate_within_72hr']==1], df_sampling_non_tenth])

In [56]:
oversampler = SMOTE(k_neighbors=5, random_state=321)

features = df_sampling_derate_tenth.loc[:, 'Aftertreatment System':].columns.tolist()

target = ['derate_within_72hr']

X_train = df_sampling_derate_tenth[features]
y_train = df_sampling_derate_tenth[target]

In [57]:
X_smote, y_smote = oversampler.fit_resample(X_train, y_train)

In [58]:
smote_all = pd.concat([y_smote, X_smote], axis=1)
smote_derate = smote_all[smote_all['derate_within_72hr']==1]
smote_derate.shape

(18855, 56)

In [59]:
und_samp = pd.concat([smote_derate, df_sampling[df_sampling['derate_within_72hr']==0]])
und_samp.shape

(213477, 56)

In [60]:
undersampler = RandomUnderSampler(random_state=321, sampling_strategy=1/2)

features = und_samp.loc[:, 'Aftertreatment System':].columns.tolist()

target =['derate_within_72hr']

X_train_final = und_samp[features]
y_train_final = und_samp[target]

X_test = obs_72_test[features]
y_test = obs_72_test[target]

In [61]:
X_resampled, y_resampled = undersampler.fit_resample(X_train_final, y_train_final)

In [62]:
print(y_resampled.shape)

(56565, 1)


In [63]:
rf_pipeline = Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('rf_clf', RandomForestClassifier(n_estimators = 500, n_jobs = -1, random_state = 69))
    ]
)

In [64]:
rand_forest = rf_pipeline.fit(X_resampled, y_resampled)
rf_pipeline.score(X_test, y_test)

y_pred = rand_forest.predict(X_test)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


In [65]:
data = {'feature': np.array(X_resampled.columns), 'importance': rf_pipeline.named_steps['rf_clf'].feature_importances_}
pd.DataFrame(data).sort_values('importance', ascending = False)


,feature,importance
29,Engine Protection Torque Partial Derate,2.604006e-01
0,Aftertreatment System,2.070301e-01
51,Unknown,1.526983e-01
18,Engine Coolant,9.990246e-02
22,Engine Fuel,5.512731e-02
43,J1939 Network,4.071594e-02
28,Engine Protection Torque Full Derate,2.484526e-02
41,Intake Manifold,2.254798e-02
17,Engine Control Module,2.108546e-02
52,Variable Geometry Turbocharger,2.051860e-02


In [66]:
confusion_matrix(y_test, y_pred)

array([[189581,   7270],
       [  2445,   3453]], dtype=int64)

In [67]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [68]:
(tn, fp, fn, tp)

(189581, 7270, 2445, 3453)

In [69]:
print(classification_report(y_test, y_pred, zero_division = 0))

              precision    recall  f1-score   support

         0.0       0.99      0.96      0.98    196851
         1.0       0.32      0.59      0.42      5898

    accuracy                           0.95    202749
   macro avg       0.65      0.77      0.70    202749
weighted avg       0.97      0.95      0.96    202749



In [122]:
lg_pipeline =  Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('lg_clf', LogisticRegression())
    ]
)

In [124]:
logistic = lg_pipeline.fit(X_resampled, y_resampled)

y_pred2 = logistic.predict(X_test)

y_pred_proba = logistic.predict_proba(X_test)[::,1]

auc = metrics.roc_auc_score(y_test, y_pred_proba)

print(auc)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9211188136613722
